In [1]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam, SGD, Nadam, RMSprop
from keras.preprocessing import image

Using TensorFlow backend.


In [3]:
# Load weights of convolution layers
from keras import backend as K
vgg_model = VGGFace(include_top=False,weights='vggface', input_shape=(200,200,3))
print('CNN Weights Loaded.')

CNN Weights Loaded.


In [10]:
#Training FC layers & finetuning lower CNN layers while freezing top most CNN layers
nb_class = 4
hidden_dim = 512

last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

# Freezing the first 5 CNN layers 
layer_count = 0
for layer in custom_vgg_model.layers:
    layer_count=layer_count+1
    if(layer_count<8):
        custom_vgg_model.layers[layer_count].trainable=False
    
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 200, 200, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 200, 200, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 100, 100, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 100, 100, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 100, 100, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 50, 50, 128)       0         
__________

In [11]:
# Loading train, test and validation data
train_data_dir = '/Users/seba/desktop/Datasets/YaleDataset/train'
validation_data_dir = '/Users/seba/Desktop/Datasets/YaleDataset/validate'
#test_data_dir = '/Users/seba/Desktop/YaleDataset/test'
nb_train_samples = 475
nb_validation_samples = 58
nb_test_samples = 52
img_height = 200
img_width = 200 
batch_size = 16

In [12]:
# prepare data augmentation configuration
validate_datagen = ImageDataGenerator(
    preprocessing_function=utils.preprocess_input,
    featurewise_center=True,
    featurewise_std_normalization=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_datagen = ImageDataGenerator(
    preprocessing_function=utils.preprocess_input,
    featurewise_center=True, 
    featurewise_std_normalization=True,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 1900 images belonging to 4 classes.
Found 232 images belonging to 4 classes.


In [13]:
# Compile model 
custom_vgg_model.compile(optimizer=SGD(lr=0.001, momentum=0.7, decay=1e-4), loss='categorical_crossentropy',metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,patience=0, min_lr=1e-6)
tensorboard_callback=TensorBoard(log_dir='/Users/seba/Desktop/Face Recognition/Logs/log_1/', histogram_freq=0,write_graph=True,write_images=False)


In [ ]:
#Training FC layers & finetuning lower CNN layers while freezing top most CNN layers
custom_vgg_model.fit_generator( 
    train_generator,
    steps_per_epoch=nb_train_samples/batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/batch_size,callbacks=[tensorboard_callback])

Epoch 1/10


/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:514: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


29/29 [==============================] - 1072s - loss: 11.6775 - acc: 0.2586 - val_loss: 12.7602 - val_acc: 0.2083

In [ ]:
# Train last FC layer or train whole network including CNN layers? 
# Validation_steps??

#The number of samples processed for each epoch is batch_size * steps_per_epochs.
#steps_per_epoch = number of times weights are updated per epoch